# Monte Carlo Options

 This is an attempt at pricing Call options using a Monte Carlo simulation and the Black Scholes Formula

## Import some libraries

In [0]:
# standard data science toolkit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from math import sqrt, exp

# useful to properly order columns in pandas DataFrame
from collections import OrderedDict

In [0]:
# some visualization boilerplate
pd.options.display.max_rows = 10
pd.options.display.max_columns = 10
pd.options.display.float_format = '{:.4f}'.format
pd.options.mode.chained_assignment = None 

In [50]:
# install Google Finance library
!pip install googlefinance.client

In [0]:
# function from the Google Finance lib that allows us to retrieve stock prices
from googlefinance.client import get_price_data

## Set constants

Here, we'll set the annual risk free rate as a constant

In [0]:
RISK_FREE_RATE = 1.17 / 100 # found somewhere on the internet

## Retrieve stock data
We'll do this using the Google Finance get_price_data A.P.I

In [51]:
def get_stock(symbol, exchange):
  '''
  inputs:
    symbol: str, representing the symbol of the stock (ex: 'GOOG')
    exchange: str, representing the exchange on which the stock is traded (ex: 'NASDAQ')
  output:
    stock Pandas DataFrame with columns 'Close' and 'Returns'
  '''
  stock = get_price_data({
    'q': symbol,
    'i': "86400", # Interval size in seconds ("86400" = 1 day intervals)
    'x': exchange,
    'p': "1Y" # Period (Ex: "1Y" = 1 year)  
  })[['Close']]
  stock['Returns'] = stock['Close'].pct_change()
  return stock

def get_stock_annualized_volatility(stock):
  '''
  inputs:
    stock: stock DataFrame
  returns:
    float representing stock annualized volatility
  '''
  return stock['Returns'].std() * sqrt(252)

# little example of the usage of the 2 functions above
goog = get_stock('GOOG', 'NASDAQ')
vol = get_stock_annualized_volatility(goog)

print("Annualized volatility:", vol)
goog.head()

('Annualized volatility:', 0.19461809848158695)


,Close,Returns
2017-03-14 20:00:00,845.6200,nan
2017-03-15 20:00:00,847.2000,0.0019
2017-03-16 20:00:00,848.7800,0.0019
2017-03-17 20:00:00,852.1200,0.0039
2017-03-20 20:00:00,848.4000,-0.0044


## Now let's dive into the Monte Carlo simulation
We'll use the Black Scholes formula to simulate the stock value after a certain period

In [0]:
def black_scholes(S, r, sig, Tmt, n):
  '''
  inputs:
    S: float, the underlying stock price
    r: float, the annualized risk free rate
    sig: float, the annualized volatility of the underlying stock
    Tmt: float, "T minus t" meaning the remaining time before the expiry date,
      expressed as a % of the year
    n: number of outcomes to simulate
   output:
    numpy.array of shape (n,) containing the n simulations
  
  '''
  return (S * np.exp((r - 0.5 * sig**2) * Tmt + sig * sqrt(Tmt) * 
                     np.random.standard_normal(n)))

In [0]:
def simulate(
    stock_price, volatility,
    end_date, risk_free_rate,
    strike_price, nb_simulations):
  '''
  inputs:
    stock_price: float representing the underlying price
    volatility: annualized stock volatility
    end_date: datetime representing expiry date (format: '2018-03-14')
    risk_free_rate: float representing risk free interest rate to discount by
    strike_price: float representing the price of the strike
    nb_simulation: integer representing the number of simulations to run
  
  output:
    simulation pd.DataFrame with columns 'Asset price', 'Payoff' and 
      'Discounted payoff'
  '''
  remaining_time = (datetime.strptime(end_date, '%Y-%m-%d') - 
                    datetime.now()).days / 365.0
  discount_factor = exp(-risk_free_rate * remaining_time)
  simulated_prices = black_scholes(stock_price, risk_free_rate, 
                                   volatility, remaining_time, nb_simulations)
  payoffs = (simulated_prices - strike_price).clip(0)
  discounted_payoffs = payoffs * discount_factor
  return pd.DataFrame(OrderedDict([
    ('Asset price', simulated_prices),
    ('Payoff', payoffs),
    ('Discounted payoff', discounted_payoffs)
  ]))

We can now automatically retrieve historical data for a stock, compute its volatility and run simulations for a given strike price and expiry date.

By averaging the discounted payoff, I can get a price estimate for the Call option.

In [0]:
def price_option(symbol, exchange, end_date, strike_price, nb_simulations):
  stock = get_stock(symbol, exchange)
  S = stock['Close'][-1]
  vol = get_stock_annualized_volatility(stock)
  simulation = simulate(S, vol, end_date, RISK_FREE_RATE, 
                        strike_price, nb_simulations)
  return simulation['Discounted payoff'].mean()

## Let's try it out

I found on Yahoo Finance an option for Apple expiring on April 13th 2018 with a strike price of 135 valued at 48.3.

I can run a million simulations to try and find out if the option is over-priced or under-priced.

In [64]:
symbol = 'AAPL'
exchange = 'NASDAQ'
expiry_date = '2018-04-13'
strike_price = 135

nb_simulations = 1000000

estimated_price = price_option(symbol, exchange, expiry_date, 
                               strike_price, nb_simulations)

print("Underlying Stock:", symbol)
print("Option expiry date:", expiry_date)
print("Option strike price:", strike_price)
print("Estimated price:", estimated_price)

('Underlying Stock:', 'AAPL')
('Option expiry date:', '2018-04-13')
('Option strike price:', 135)
('Estimated price:', 45.085469518351616)
